# First Modeling

In [ ]:
## Import all the libraries
import pandas as pd
import numpy as np

## Data Viz 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

## Transformation
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

## for statistical tests
from math import sqrt
import scipy
from scipy.fft import fft, fftfreq
import statistics
from statistics import mean
import statsmodels.api as sm
import statsmodels.formula.api as smf

## SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler,  ClusterCentroids
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score
from sklearn.svm import SVC

## Cluster Centroids
from imblearn.under_sampling import ClusterCentroids
from sklearn.cluster import KMeans

## Modelling 
from sklearn import datasets, decomposition, ensemble, feature_selection, linear_model, metrics, model_selection, preprocessing, svm, tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.feature_selection import SelectFromModel, SelectKBest
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix, explained_variance_score, f1_score, mean_absolute_error, mean_squared_error, precision_score, r2_score, recall_score, roc_curve, roc_auc_score, precision_recall_curve, average_precision_score
from sklearn.model_selection import cross_validate, cross_val_predict, cross_val_score, GridSearchCV, ShuffleSplit, train_test_split
from sklearn.calibration import calibration_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC, SVR
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import xgboost as xgb

import shap

from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
import time

print('Libraries imported successfully')

In [ ]:
# Load Code for Kaggle
df = pd.read_csv('/kaggle/input/arrhythmia-preprocessed/arrhythmia_preprocessed_cleaned_classes_label(1).csv', sep=',', index_col=0)
print('Dataset imported successfully')

In [ ]:
# Load Code for GitHub
# df = pd.read_csv('arrhythmia_preprocessed_cleaned_classes_label.csv')

# Seperate Features, Standardize and Split

In [ ]:
# Separate features and target variable
X = df.drop(['class','label'], axis=1)  # Features
y = df['label']  # Target variable

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Save X train and X_test without PCA 
X_train_saved = X_train
X_test_saved = X_test
y_train_saved = y_train
y_test_saved = y_test

## PCA - Principal Component Analysis

In [ ]:
# Define different numbers of components to try
n_components_list = [30,40,50,55,60,65,70,75,100]
cumulative_variance_ratios = []
for n_components in n_components_list:
    
    # Instantiate PCA with desired number of components
    pca = PCA(n_components=n_components)
    
    # Fit PCA to the standardized data
    pca.fit(X_scaled)
    
    # Transform the data into the new feature space
    X_pca = pca.transform(X_scaled)
    
    # Save the results in individual dataframes
    globals()[f'X_pca_{n_components}'] = pd.DataFrame(X_pca)
    
    # Get the explained variance ratio
    explained_variance_ratio = pca.explained_variance_ratio_
    cumulative_variance_ratio = sum(explained_variance_ratio)
    cumulative_variance_ratios.append(cumulative_variance_ratio)
    
    # Print cumulative variance ratio
    print(f'Cumulative variance ratio with {n_components} components:', cumulative_variance_ratio)

# Plot cumulative variance ratio
plt.figure(figsize=(8, 6))
plt.plot(n_components_list, cumulative_variance_ratios, marker='o', linestyle='-')
plt.title('Cumulative Variance Ratio vs. Number of Components', fontsize=14)
plt.xlabel('Number of Components', fontsize=14)
plt.ylabel('Cumulative Variance Ratio', fontsize=14)
plt.xticks(n_components_list)
plt.grid(True)

# Add annotations
for i, txt in enumerate(cumulative_variance_ratios):
    plt.annotate(f'{txt:.2f}', (n_components_list[i], cumulative_variance_ratios[i]), textcoords="offset points", xytext=(0,10), ha='center')
plt.tight_layout()
plt.savefig('PCA_cumulative_variance_ratio_vs_nr_components.png')
plt.show()

In [ ]:
# Re-run PCA with the selected number of components 
pca = PCA(n_components=.9)
X_pca = pca.fit_transform(X_scaled)

# Get the explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

# Visualize the explained variance ratio
plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, alpha=0.8)
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance Ratio for Principal Components')
plt.show()

# Setting the PCA Threshold for the Training & Test Datasets

In [ ]:
# PCA on training data set 
pca = PCA(n_components = .2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

print(pca.n_components_)

In [ ]:
# Initialize PCA with 'mle' to automatically determine the number of components
#pca = PCA(n_components='mle')
# Fit PCA to the standardized data
#pca.fit(X_scaled)
# Get the number of components that explain at least 90% of the variance
#n_components = pca.explained_variance_ratio_.cumsum().searchsorted(0.90) + 1
# Print the selected number of components
#print(f'Selected number of components: {n_components}')

In [ ]:
#SMOTE
#smo = SMOTE()
#X_train_sm, y_train_sm = smo.fit_resample(X_train, y_train)

#print("Shape of X_train resampled with smote:", X_train_sm.shape)
#print("Shape of y_train resampled with smote:", y_train_sm.shape)
#print('SMOTE :', dict(pd.Series(y_train_sm).value_counts()))

# Set X_train and X_test 

In [ ]:
X_train = X_train_pca
X_test = X_test_pca
print(X_train.shape)
print(X_test.shape)

# Model Testing

# Define Classifiers and parameters

In [ ]:
# Define classifiers with specified parameters
clf_lr = LogisticRegression(random_state=22, max_iter=2000)
clf_rf = RandomForestClassifier(random_state=22)
clf_svc = SVC(random_state=22)
clf_en = LogisticRegression(penalty='elasticnet', solver='saga', max_iter=10000)
clf_gb = GradientBoostingClassifier(random_state=42)
clf_ada = AdaBoostClassifier()
clf_xgb = xgb.XGBClassifier()


# Define parameter grids for each classifier
param_grid_lr = [{'C': [c], 'penalty': [penalty]} for c in np.logspace(-4, 2, 9) for penalty in ['l1', 'l2']]

param_grid_rf = [{'n_estimators': [10, 50, 100, 250, 500, 1000], 
                  'min_samples_leaf': [1, 3, 5], 
                  'max_features': ['sqrt', 'log2']}]

param_grid_svc = {'C': np.logspace(-4, 2, 9), 'kernel': ['linear', 'rbf']}
param_grid_svc_list = [{'C': [c], 'kernel': [kernel]} for c in np.logspace(-4, 2, 9) for kernel in ['linear', 'rbf']]

param_grid_en = {'C': np.logspace(-4, 2, 9), 'l1_ratio': np.linspace(0.1, 0.9, 9)}
param_grid_en_list = [{'C': [C], 'l1_ratio': [l1_ratio]} for C in np.logspace(-4, 2, 9) for l1_ratio in np.linspace(0.1, 0.9, 9)]


param_grid_gb = {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 1.0], 'max_depth': [3, 5, 7]}
param_grid_gb_list = [{'n_estimators': [n_estimators], 'learning_rate': [learning_rate], 'max_depth': [max_depth]} 
                                 for n_estimators in [50, 100, 200] 
                                 for learning_rate in [0.01, 0.1, 1.0] 
                                 for max_depth in [3, 5, 7]]

param_grid_ada = {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 1.0]}
param_grid_ada_list = [{'n_estimators': [n_estimators], 'learning_rate': [learning_rate]} 
                       for n_estimators in [50, 100, 200] 
                       for learning_rate in [0.01, 0.1, 1.0]]

param_grid_xgb = {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 1.0], 'max_depth': [3, 5, 7]}
param_grid_xgb_list = [{'n_estimators': [n_estimators], 'learning_rate': [learning_rate], 'max_depth': [max_depth]} 
                       for n_estimators in [50, 100, 200] 
                       for learning_rate in [0.01, 0.1, 1.0] 
                       for max_depth in [3, 5, 7]]

print('parameters set')

# Model Training Function with Progress Bar and Score Output

In [ ]:
# Define train_model function
def train_model(clf, param_grid, X_train, y_train, X_test, y_test):
    # Initialize GridSearchCV with the classifier and parameter grid
    gcv = GridSearchCV(clf, param_grid, cv=3, refit=True)

    # Calculate total number of iterations for progress bar
    total_iterations = len(param_grid) * 3

    # Train the model with tqdm progress bar
    with tqdm(total=total_iterations, desc="Training Progress") as pbar:
        for params in param_grid:
            for fold in range(3):
                # Update progress bar
                pbar.update(1)

                # Set parameters individually
                for param_name, param_value in params.items():
                    setattr(clf, param_name, param_value)

                # Train the model
                gcv.fit(X_train, y_train)

    # Make predictions
    train_predictions = gcv.predict(X_train)
    test_predictions = gcv.predict(X_test)

    # Calculate evaluation metrics
    train_acc = accuracy_score(y_train, train_predictions)
    test_acc = accuracy_score(y_test, test_predictions)
    train_precision = precision_score(y_train, train_predictions)
    test_precision = precision_score(y_test, test_predictions)
    train_recall = recall_score(y_train, train_predictions)
    test_recall = recall_score(y_test, test_predictions)
    train_f1 = f1_score(y_train, train_predictions)
    test_f1 = f1_score(y_test, test_predictions)

# Calculate AUROC scores
    train_auroc = roc_auc_score(y_train, train_predictions)
    test_auroc = roc_auc_score(y_test, test_predictions)     

# Return model performance metrics
    return {
        'model': type(clf).__name__,
        'best_params': gcv.best_params_,
        'train_accuracy': train_acc,
        'test_accuracy': test_acc,
        'train_precision': train_precision,
        'test_precision': test_precision,
        'train_recall': train_recall,
        'test_recall': test_recall,
        'train_f1': train_f1,
        'test_f1': test_f1,
        'train_auroc': train_auroc,
        'test_auroc': test_auroc,
        'test_predictions': test_predictions
    }

print('function defined')

In [ ]:
# Define a function to print the results in a readable format
def print_results(results):
    print(f"Model: {results['model']}")
    print(f"Best Parameters: {results['best_params']}")
    print("Training Metrics:")
    print(f"  Accuracy: {results['train_accuracy']:.2f}")
    print(f"  Precision: {results['train_precision']:.2f}")
    print(f"  Recall: {results['train_recall']:.2f}")
    print(f"  F1-score: {results['train_f1']:.2f}")
    print(f"  Auroc-score: {results['train_auroc']:.2f}")    
    print("Test Metrics:")
    print(f"  Accuracy: {results['test_accuracy']:.2f}")
    print(f"  Precision: {results['test_precision']:.2f}")
    print(f"  Recall: {results['test_recall']:.2f}")
    print(f"  F1-score: {results['test_f1']:.2f}")
    print(f"  Auroc-score: {results['test_auroc']:.2f}")    
    print()

print('function defined')

# FYI for reffering back to scores later: 
# Access logistic regression recall score for testing
#lr_test_recall = lr_results['test_recall']
# Now you can use lr_test_recall for further analysis or comparison
#print("Logistic Regression Recall Score for Testing:", lr_test_recall)

# Logistic Regression

In [ ]:
# run the train model function for lr
lr_results = train_model(clf_lr, param_grid_lr, X_train, y_train, X_test, y_test)
# print the results
print_results(lr_results)

# Random Forest

In [ ]:
# run the train model function for rf
rf_results = train_model(clf_rf, param_grid_rf, X_train, y_train, X_test, y_test)
# print the results
print_results(rf_results)

# SVM

In [ ]:
# run the train model function for svc
svc_results = train_model(clf_svc, param_grid_svc_list, X_train, y_train, X_test, y_test)
# print the results
print_results(svc_results)

# ElasticNet

In [ ]:
# run the train model function for elasticnet
en_results = train_model(clf_en, param_grid_en_list, X_train, y_train, X_test, y_test)
# print the results
print_results(en_results)

# Gradientboost

In [ ]:
# run the train model function for gradientboost
gb_results = train_model(clf_gb, param_grid_gb_list, X_train, y_train, X_test, y_test)
# print the results
print_results(gb_results)

# Adaboost

In [ ]:
# run the train model function for adaboost
ada_results = train_model(clf_ada, param_grid_ada_list, X_train, y_train, X_test, y_test)
# print the results
print_results(ada_results)

# XGBoost

In [ ]:
# run the train model function for xgboost
xgb_results = train_model(clf_xgb, param_grid_xgb_list, X_train, y_train, X_test, y_test)
# print the results
print_results(xgb_results)

# Model Comparison

In [ ]:
# Define model names and their corresponding results
models = ['Log Regression', 'Random Forest', 'SVM', 'ElasticNet', 'AdaBoost', 'GradientBoost', 'XGBoost']
results = [lr_results, rf_results, svc_results, en_results, ada_results, gb_results, xgb_results]

In [ ]:
# Extract test and train accuracy scores for each model
test_accuracies = [result['test_accuracy'] for result in results]
train_accuracies = [result['train_accuracy'] for result in results]

# Extract test and train recall scores for each model
test_recalls = [result['test_recall'] for result in results]
train_recalls = [result['train_recall'] for result in results]

# Bar width
bar_width = 0.2
index = np.arange(len(models))

# Plotting
plt.figure(figsize=(12, 8))

# Plot test accuracy
plt.bar(index, test_accuracies, bar_width, color='lightblue', edgecolor='black', hatch='/', label='Test Accuracy')
# Plot train accuracy
plt.bar(index + bar_width, train_accuracies, bar_width, color='lightgreen', edgecolor='black', hatch='\\', label='Train Accuracy')

# Plot test recall
plt.bar(index + 2*bar_width, test_recalls, bar_width, color='lightcoral', edgecolor='black', hatch='x', label='Test Recall')
# Plot train recall
plt.bar(index + 3*bar_width, train_recalls, bar_width, color='lightyellow', edgecolor='black', hatch='.', label='Train Recall')

plt.xlabel('Model')
plt.ylabel('Scores')
plt.title('Comparison of Model Performances')
plt.xticks(index + 1.5*bar_width, models)
plt.legend()
plt.show()

# AUROC for best performing models

In [ ]:
# Define model names and their corresponding results
models = ['Log Regression', 'Random Forest', 'SVM', 'ElasticNet', 'AdaBoost', 'GradientBoost', 'XGBoost']
results = [lr_results, rf_results, svc_results, en_results, ada_results, gb_results, xgb_results]

In [ ]:
# Define model names and their corresponding results
models = ['Log Regression', 'Random Forest', 'SVM', 'ElasticNet', 'AdaBoost', 'GradientBoost', 'XGBoost']
results = [lr_results, rf_results, svc_results, en_results, ada_results, gb_results, xgb_results]


# Plotting ROC curves for each model
plt.figure(figsize=(10, 8))

for model_name, result in zip(models, results):
    # Compute ROC curve for test data
    fpr, tpr, _ = roc_curve(y_test, result['test_predictions'])
    
    # Plot ROC curve
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {result["test_auroc"]:.2f})')

# Plot ROC curve for random guessing (baseline)
plt.plot([0, 1], [0, 1], linestyle='--', color='black', label='Random Guessing')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
###

In [ ]:
# Initialize variables to keep track of the highest recall value and the corresponding model
max_recall = 0.0
best_model = None

# Iterate through each model's result
for result in results:
    model_name = result['model']
    test_recall = result['test_recall']
    
    # Check if the current model has higher recall than the previous maximum
    if test_recall > max_recall:
        max_recall = test_recall
        best_model = model_name

print(f"The model with the highest recall is {best_model} with a recall of {max_recall:.3f}")

In [ ]:
##

In [ ]:
pca = PCA(n_components=.99)
    
# Define the pipeline
pipeline = Pipeline([
    ('pca', PCA()),
    ('log_reg', LogisticRegression())
])

# Define parameter grid for PCA and logistic regression
param_grid = {
    'pca__n_components': range(1, min(X_train.shape[1], 200)),  # Adjust range as needed
    # Add any other hyperparameters for Logistic Regression if needed
    # 'log_reg__C': [0.1, 1, 10],  # Example hyperparameter for Logistic Regression
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='recall')
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best parameters:", best_params)
print("Best cross-validation accuracy:", best_score)

In [ ]:
plt.figure(figsize=(10, 6))

# Perform grid search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the results
results = grid_search.cv_results_
n_components = param_grid['pca__n_components']
mean_scores = results['mean_test_score']
std_scores = results['std_test_score']

# Plot results
plt.errorbar(n_components, mean_scores, yerr=std_scores, fmt='-o')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cross-validation Accuracy')
plt.title('Cross-validation Accuracy vs. Number of Principal Components')
plt.xticks(range(0, len(n_components), 10), rotation=90)
plt.subplots_adjust(wspace=0.5)
plt.grid(True)
plt.show()


In [ ]:
# Extract the final model (Logistic Regression) from the pipeline
log_reg_model = pipeline.named_steps['log_reg']

# Ensure that the logistic regression model is fitted with training data
log_reg_model.fit(X_train, y_train)

# Initialize a SHAP explainer with the logistic regression model
explainer = shap.Explainer(log_reg_model, X_train)

# Calculate SHAP values for the testing set
shap_values = explainer(X_test)

# Plot the SHAP summary plot
shap.summary_plot(shap_values, X_test, feature_names=X.columns.tolist(), class_names=y.unique())